In [129]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime

In [117]:
response = requests.get('https://coinmap.org/api/v1/venues/')

In [118]:
if response.status_code == 200:
    print('Success!')
elif response.status_code == 404:
    print('Not Found.')

Success!


In [119]:
response_json = response.json()
venues_array = response_json['venues']
print(len(venues_array))

22787


In [120]:
print(venues_array[0])

{'id': 8225, 'lat': 50.1220639499942, 'lon': -5.53289830684662, 'category': 'atm', 'name': 'One And All', 'created_on': 1465943680, 'geolocation_degrees': '5°31\'58.43"W, 50°7\'19.43"N'}


In [127]:

southernmost_lat = 18.464825 # southern tip of Hawaii
easternmost_lon = -66.949471 # eastern tip of Maine 

# This filtering step reduces the number of records from 22787 to 7379
# This still means we have to call the API 7379 times. Unfortunately 
# there are no further simple optimization we can make, since it turns
# out that 6430 of the 7379 actually lie within the United States 
venues_array = list(filter( lambda ven: int(ven['lon']) <= easternmost_lon, venues_array ))
venues_array = list(filter( lambda ven: int(ven['lat']) >= southernmost_lat, venues_array ))

print(len(list(venues_array)))

venues_usa = []

for venue in venues_array: 
    
    lat_param = venue['lat']
    lon_param = venue['lon']
    
    # Get response and convert it into a json
    response = requests.get(f'https://geo.fcc.gov/api/census/block/find?latitude={lat_param}&longitude={lon_param}&format=json')
    
    if response.status_code == 200:
        data = response.json()
        
        # We cast the state name and county name to string becuase when the coordinates lie
        # outside of the United States, these values are returned as type <class 'NoneType'>.
        # In this case, we simply skip this record and move on to the next on in the veneus_array.
        if str(data['State']['name']) == 'None' or str(data['County']['name']) == 'None':
            continue
        
        # If the venue is in the United States, we assign the state name and country name to the
        # current venue record. We then append this record, which now contains fields for state and
        # county to the venues_usa list. Casting to a string here is a defensive operation. 
        else:
            venue['state'] = str(data['State']['name'])
            venue['county'] = str(data['County']['name'])
            venues_usa.append(venue)
    else:
        print(response.status_code, response)
    
print(len(venues_usa))    

7379
6430


In [137]:
# Since the continuous process of calling the API for each record is so expensive
# we will want to save the data as soon as possible. 
df_venues_usa = pd.json_normalize(venues_usa)

df_venues_usa['created_on'] = pd.to_datetime(df_venues_usa['created_on'],unit='s')

df_venues_usa['year'] = df_venues_usa['created_on'].dt.year

df_venues_usa

,id,lat,lon,category,name,created_on,geolocation_degrees,state,county,promoted,year
0,8,39.261575,-121.016567,default,SierraHosts.net,2013-12-11 20:46:10,"121°0'59.64""W, 39°15'41.67""N",California,Nevada,NaN,2013
1,10,39.714651,-104.936268,default,Costa Rica Travel Market LLC,2014-01-11 13:38:47,"104°56'10.57""W, 39°42'52.74""N",Colorado,Denver,NaN,2014
2,12,42.362679,-71.086398,shopping,MIT Coop,2014-09-05 01:18:06,"71°5'11.03""W, 42°21'45.65""N",Massachusetts,Middlesex,NaN,2014
3,6,37.770518,-122.450413,default,Deborah Hodgson Vamp Couture,2013-11-28 19:29:27,"122°27'1.49""W, 37°46'13.87""N",California,San Francisco,NaN,2013
4,8226,40.752564,-86.346627,shopping,Stack-Tech Computer Repair,2016-06-15 02:03:35,"86°20'47.86""W, 40°45'9.23""N",Indiana,Cass,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...
6425,49485,35.373038,-96.930319,atm,Vapor 100 - Shawnee,2021-03-03 14:46:52,"96°55'49.15""W, 35°22'22.94""N",Oklahoma,Pottawatomie,True,2021
6426,190641,47.313752,-122.303162,atm,Vapor Cloud,2021-06-08 13:25:51,"122°18'11.38""W, 47°18'49.51""N",Washington,King,True,2021
6427,128530,32.787608,-117.238809,default,Greenfield Educational Resources,2021-04-27 20:41:28,"117°14'19.71""W, 32°47'15.39""N",California,San Diego,NaN,2021
6428,115524,33.245510,-111.794510,shopping,Physical Art For Crypto,2021-04-18 01:05:38,"111°47'40.24""W, 33°14'43.84""N",Arizona,Maricopa,NaN,2021


In [134]:
df_venues_usa

,id,lat,lon,category,name,created_on,geolocation_degrees,state,county,promoted
0,8,39.261575,-121.016567,default,SierraHosts.net,1970-01-17 01:13:14.770,"121°0'59.64""W, 39°15'41.67""N",California,Nevada,NaN
1,10,39.714651,-104.936268,default,Costa Rica Travel Market LLC,1970-01-17 01:57:27.527,"104°56'10.57""W, 39°42'52.74""N",Colorado,Denver,NaN
2,12,42.362679,-71.086398,shopping,MIT Coop,1970-01-17 07:37:59.886,"71°5'11.03""W, 42°21'45.65""N",Massachusetts,Middlesex,NaN
3,6,37.770518,-122.450413,default,Deborah Hodgson Vamp Couture,1970-01-17 00:54:26.967,"122°27'1.49""W, 37°46'13.87""N",California,San Francisco,NaN
4,8226,40.752564,-86.346627,shopping,Stack-Tech Computer Repair,1970-01-17 23:12:36.215,"86°20'47.86""W, 40°45'9.23""N",Indiana,Cass,NaN
...,...,...,...,...,...,...,...,...,...,...
6425,49485,35.373038,-96.930319,atm,Vapor 100 - Shawnee,1970-01-19 16:33:02.812,"96°55'49.15""W, 35°22'22.94""N",Oklahoma,Pottawatomie,True
6426,190641,47.313752,-122.303162,atm,Vapor Cloud,1970-01-19 18:52:38.751,"122°18'11.38""W, 47°18'49.51""N",Washington,King,True
6427,128530,32.787608,-117.238809,default,Greenfield Educational Resources,1970-01-19 17:52:36.088,"117°14'19.71""W, 32°47'15.39""N",California,San Diego,NaN
6428,115524,33.245510,-111.794510,shopping,Physical Art For Crypto,1970-01-19 17:38:27.938,"111°47'40.24""W, 33°14'43.84""N",Arizona,Maricopa,NaN


In [138]:
df_venues_usa.to_csv('../../data/interim/CRYTO_VENUES_USA', index=False,header=False)